In [5]:
from gridworld import *
from tqdm import trange
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#Normal Q-learning

gw = gridworld((5, 5), (0, 4), (4, 4), [(o, 4) for o in range(1, 4)], 0.9, 0, 1, 0)
y = 0.95

In [8]:
#Empirically estimate risk probability
num_sims = 10000
def estimate_rr(pi):
    risky = 0
    rewards = []
    discounted = []
    for i in trange(num_sims):
        s = gw.reset()
        reward = 0
        d = False
        j = 0
        while(j < 1000):
            a = pi[s[0],s[1]]
            s1,r,d = gw.step(a)
            reward += r
            if d == True:
                if s1 in gw.error_states:
                    risky += 1
                    discounted_reward = 0
                else:
                    discounted_reward = pow(y, j) * r
                break
            s = s1
            j += 1
        rewards.append(reward)
        discounted.append(discounted_reward)
    return sum(discounted)/float(num_sims), sum(rewards)/float(num_sims), float(risky)/num_sims    

In [10]:
trials = 10
Q_list = []
reward_trial_list = []

np.random.seed(1)

# Set learning parameters
lr = 0.9
num_episodes = 100000

for k in range(trials):
    #Initialize table with all zeros
    Q = np.zeros((gw.M,gw.N,4))
    jList = []
    rList = []
    for i in trange(num_episodes):
        #print('Episode', i)
        #Reset environment and get first new observation
        s = gw.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Table learning algorithm
        while j < 1000:
            j+=1
            #Choose an action by greedily (with noise) picking from Q table
            a = np.argmax(Q[s[0],s[1],:] + np.random.randn(1,4)*(1./(i+1)))
#             if np.random.rand() < epsilon:
#                 # take random action
#                 a = np.random.choice([0,1,2,3])
#             else:
#                 # take action according to the q function table
#                 a = np.argmax(Q[s[0],s[1],:])
            #Get new state and reward from environment
            s1,r,d = gw.step(a)

            #Update Q-Table with new knowledge
            Q[s[0], s[1],a] = Q[s[0], s[1],a] + lr*(r + y*np.max(Q[s1[0], s1[1],:]) - Q[s[0], s[1],a])

            rAll += r
            s = s1
            if d == True:
                break
        jList.append(j)    
        rList.append(rAll)
        
    Q_list.append(Q)
#     v = np.mean(Q, axis = 2)
#     gw.print_v(v)

    pi = np.argmax(Q, axis = 2)
    gw.print_policy(pi)
#     reward_trial_list.append(sum(rList)/float(num_episodes))
#     print "Score over time: " +  str(sum(rList)/float(num_episodes))

#     window = int(num_episodes/10)

#     plt.figure(figsize=[9,16])
#     plt.subplot(411)
#     plt.plot(pd.Series(jList).rolling(window).mean())
#     plt.title('Step Moving Average ({}-episode window)'.format(window))
#     plt.ylabel('Moves')
#     plt.xlabel('Episode')

#     plt.subplot(412)
#     plt.plot(pd.Series(rList).rolling(window).mean())
#     plt.title('Reward Moving Average ({}-episode window)'.format(window))
#     plt.ylabel('Reward')
#     plt.xlabel('Episode')

#     plt.tight_layout(pad=2)
#     plt.show()

gw.print_gw()
meanQ = np.mean(np.array(Q_list), axis = 0)
meanv = np.mean(meanQ, axis = 2)
gw.print_v(meanv)
meanpi = np.argmax(meanQ, axis = 2)
gw.print_policy(meanpi)
E_dis_reward, E_reward, E_risk = estimate_rr(meanpi)
print "Mean reward (empirical)", E_dis_reward
print "Successful Episodes ", E_reward
print "Risk probab ", E_risk

100%|██████████| 100000/100000 [00:20<00:00, 4971.62it/s]


,0,1,2,3,4
0,➡,➡,⬇,⬇,⬅
1,⬇,➡,⬇,⬇,E
2,➡,➡,➡,⬇,E
3,⬆,➡,➡,⬇,E
4,➡,⬆,➡,➡,G


100%|██████████| 100000/100000 [00:39<00:00, 2562.46it/s]


,0,1,2,3,4
0,➡,⬇,⬅,⬅,⬅
1,⬇,⬇,⬅,⬅,E
2,➡,⬇,⬇,⬆,E
3,⬇,➡,⬇,⬅,E
4,➡,⬆,➡,➡,G


,0,1,2,3,4
0,-,-,-,-,S
1,-,-,-,-,E
2,-,-,-,-,E
3,-,-,-,-,E
4,-,-,-,-,G


,0,1,2,3,4
0,0.06,0.13,0.16,0.14,0.093
1,0.11,0.13,0.15,0.18,E
2,0.14,0.11,0.21,0.18,E
3,0.15,0.16,0.19,0.19,E
4,0.09,0.17,0.18,0.25,G


,0,1,2,3,4
0,➡,⬇,⬅,⬇,⬅
1,⬇,⬇,⬇,⬇,E
2,➡,⬇,➡,⬇,E
3,⬇,➡,⬇,⬇,E
4,➡,⬆,➡,➡,G


100%|██████████| 10000/10000 [00:00<00:00, 15927.02it/s]

Mean reward (empirical) 0.638443296904
Successful Episodes  0.8662
Risk probab  0.1338
